In [ ]:
import geopandas as gpd
import pandas as pd
import shutil
import matplotlib.pyplot as plt
# import seaborn as sns
import copy
import numpy as np
import shapely
from shapely.geometry import Point
import string
import os, sys, getpass
import io, uuid
import time

# Set up

### Common folders and variables

In [ ]:
table_source = "C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/TABLES"
attributes_source = "C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet"
stad_source = "C:/Users/akm03/Alaa/NERGYHUB/data/vasteras_stad/processed_data"


min_height = 2.4
min_volume = 3.2

# common names of files that will be used
raw_buildings = 'buildings_filtered'
final_buildings = "buildings"

raw_properties = "ay_riks_updated"
final_properties = 'properties'

raw_addresses = 'ADRPL_90A.xlsx'
final_addresses = 'addresses'

height_on_perimeter = 'dtm_values_at_perimeters.csv'


#final_file = 'joined_buildings_addresses_properties'

### open attribute table that will be use to specify columns to keep and their new names
data_list = os.path.join(attributes_source, "data_list.xlsx")

attributes_sheet_name = "Attributes_description"
attributes = pd.read_excel(data_list, attributes_sheet_name)

codes_sheet_name = "Codes"
codes = pd.read_excel(data_list, codes_sheet_name)

options_sheet_name = "Attributes_options"
options = pd.read_excel(data_list, options_sheet_name)

In [ ]:
city_name = 'Vasteras'

input_dtm = 'C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/{}/DTM/at_perimeters'.format(city_name)
input_dsm = 'C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/{}/DSM/at_perimeters'.format(city_name)
input_dsm_pixel = 'C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/{}/DSM/at_buildings'.format(city_name)

destination = "C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/{0}/processed_data".format(city_name)
source = "C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/{0}/Fastighetskartan".format(city_name)

# extent = gpd.read_file(os.path.join(destination, 'properties_extent.shp'))

In [ ]:
def check_duplication(df, column = None, drop = False, **kwargs):    
    if column and column not in df.columns:
        print('Column {} does not exist in dataframe'.format(column))
    else:
        if column is None:
            column = list(df.columns)   
        df_duplicated = df.loc[df[column].duplicated(), :]
        if len(df_duplicated)>0:    
            print(len(df_duplicated[column].values.tolist()))
            #print(df_duplicated[column].values.tolist())
            if drop:
                #print(kwargs)
                df.drop_duplicates(subset = column, **kwargs)
                return df
        else:
            print("No duplications")

### Convert addresses from tables to a shapefile (already run)

In [ ]:
# # set the table source as the directory
# os.chdir(table_source)
# sheetname = 'Data'
# # read file
# addresses_table = pd.read_excel(raw_addresses, sheet_name = sheetname)
# # use deep copy to reduce loading time of the data especially when debugging the code
# addresses = copy.deepcopy(addresses_table)
# # remove rows with no value for POSTNR because it is not a building
# addresses = addresses[addresses['POSTNR'].notnull()]

# #### convert file to geopandas
# from geopandas import GeoDataFrame
# from shapely.geometry import Point

# Xcoord = 'YKOORD'
# Ycoord = 'XKOORD'
# XcoordL = 'YKOORDL'
# YcoordL = 'XKOORDL'

# geometry = [Point(xy) for xy in zip(addresses[Xcoord], addresses[Ycoord])]
# addresses = addresses.drop([Xcoord, Ycoord, XcoordL, YcoordL], axis=1)
# addresses = GeoDataFrame(addresses, crs="EPSG:3006", geometry=geometry)

# # Save as shapefile
# addresses.to_file(final_addresses + '.shp')

# Buildings data

In [ ]:
# set the destination as the directory
os.chdir(destination)

In [ ]:
#################### read the building file for curation
buildings = gpd.read_file(raw_buildings + ".shp")

In [ ]:
buildings = pd.read_excel(raw_buildings + ".xlsx")

In [ ]:
buildings.columns

In [ ]:
# calculate height as the difference between the average DTM and maximum DSM
buildings['Height'] = buildings['DSM_max'] - buildings['DTM_mean']

#### Some buildings will have less than 2.4 m height which is incorrect. That means that this method has a certain uncertainty. 

Calculate the percentage of builidngs with a height < 2.4 and set their height to nan

In [ ]:
# check values range
print(buildings[['Height']].describe())
buildLessThan2o2 = buildings.loc[buildings['Height']<min_height, 'Height']
buildLessThan2o2.hist(bins=20)
print('Number of builidngs with abnormal height : {0}'.format(len(buildLessThan2o2)))

In [ ]:
percBuildLessThan2o2 = len(buildLessThan2o2)/len(buildings)*100
print('Percentage of buildings with abnormal height :{0} %'.format(round(percBuildLessThan2o2,2)))

In [ ]:
# Set the heught of these buildings to Nan
buildings.loc[buildings['Height']<min_height, 'Height'] = np.nan

In [ ]:
len(buildings)

In [ ]:
buildings.columns

#### Delete/rename buildings shapefiles columns

In [ ]:
# get buildings attributes only
attributes_buildings = attributes[attributes['Feature']=='Buildings']
updated_columns = []
for col in buildings.columns:
    j = attributes_buildings['Attribute'].tolist().index(col)
    updated_columns.append(attributes_buildings['NewName'].tolist()[j])
drop_columns = [col for col in buildings.columns if updated_columns[buildings.columns.tolist().index(col)] == 'REMOVE']
updated_columns = [i for i in updated_columns if i != 'REMOVE']
buildings.drop(drop_columns, inplace=True, axis=1)
buildings.columns = updated_columns

In [ ]:
# replace Type values by their english synonyms for data_file, options sheet
# swedish list
swedish = options[options['Attribute']=='DETALJTYP']['Purpose'].tolist()
english = options[options['Attribute']=='DETALJTYP']['Definition'].tolist()
buildings['Type'] = buildings['Type'].replace(swedish, english)

In [ ]:
def purpose(x, codes_list, purposes_list, column_name):
    j = codes_list.index(x[column_name])
    return purposes_list[j]

Add Purpose and Details columns that give information about the building purpose based on its code

In [ ]:
# read codes list from data list, codes sheet
#code column name
code_column = 'Code'

codes_list = codes[code_column].tolist()
purposes_list = codes['Purpose'].tolist()
detailed_purposes_list = codes['Details'].tolist()
buildings['Purpose'] = buildings.apply(lambda x: purpose(x, codes_list, purposes_list, code_column), axis = 1)
buildings['Details'] = buildings.apply(lambda x: purpose(x, codes_list, detailed_purposes_list, code_column), axis = 1)

In [ ]:
# check the units of the shapefile
print(buildings.crs.axis_info[0].unit_name)
# add area field; convert if unit is not meter
buildings['Area'] = buildings.geometry.area

#### Buildings volumes
The buildings volumes to be calculated by multiplying the area with the height on the perimeter. It is calculated in 1_nrgyhub_qgis jupyter notebook. Load the resulted file to calculate the volume

In [ ]:
heightsPerimeter = pd.read_csv(os.path.join('C:/Users/akm03/Alaa/NERGYHUB/data/lantmateriet/Vasteras/DSM/las2dempro/at_deeper_perimeters', "dsm_values_at_perimeters.csv"))
#heightsPerimeter.drop(columns = ['Unnamed: 0'], inplace = True)
check_duplication(heightsPerimeter, column = 'ID', drop = True, 
                                     keep = 'first', inplace = True, ignore_index = True)

# concatenate files based on building id
buildings = buildings.merge(heightsPerimeter, how='left', left_on='BuildingID', right_on='ID')

In [ ]:
# check columns and drop redundant varibales
print(buildings.columns)

# calculate volume
buildings['Height2'] = buildings['Value']-buildings['DTM_mean']
buildings.loc[buildings['Height2']<min_height, 'Height2'] = np.nan
buildings['Volume'] = buildings['Area']*buildings['Height2']
buildings.drop(columns = ['ID', 'Value'], inplace = True)

#### Some buildings have negative values and less than 3.2 m3. If height != 0, replace volume by height*area

In [ ]:
buildings['Volume2'] = buildings['Height'] * buildings['Area']
buildings.loc[(buildings['Volume']<min_volume)&(buildings['Height']!=0), 'Volume'] = buildings.loc[(buildings['Volume']<min_volume)&(buildings['Height']!=0), 'Volume2']
buildings.loc[buildings['Volume']<min_volume, 'Volume'] = np.nan

In [ ]:
buildings.drop(columns = ['Volume2'], inplace = True)

In [ ]:
import scipy.stats as stats
import numpy as np
from scipy import stats


def freedman_diaconis(data, returnas="width"):
    data = np.asarray(data, dtype=np.float_)
    IQR  = stats.iqr(data, rng=(25, 75), scale="raw", nan_policy="omit")
    N    = data.size
    bw   = (2 * IQR) / np.power(N, 1/3)

    if returnas=="width":
        result = bw
    else:
        datmin, datmax = data.min(), data.max()
        datrng = datmax - datmin
        result = int((datrng / bw) + 1)
    return(result)

def plot_hist(ID, df):
    df = df.loc[df['ID']==ID,'Value']
    print(df.min(), df.max(), df.median(), df.mean())
    fig, ax = plt.subplots(figsize = (6,4))
    # Plot histogram
    #df.plot(kind = "hist", density = True, bins = 20, grid=True,) # change density to true, because KDE uses density
    ax.hist(df, bins=freedman_diaconis(df, returnas = 'bins'), density = True)
    # Plot KDE
    df.plot(kind = "kde")
    
    # non-parametric pdf
    nparam_density = stats.kde.gaussian_kde(df.ravel())
    x = np.linspace(-20, max(df)+20, 500)
    nparam_density = nparam_density(x)
    #ax.plot(x, nparam_density, 'r-', label='non-parametric density (smoothed by Gaussian kernel)')
    
#     # Calculate percentiles
#     quant_5, quant_25, quant_50, quant_75, quant_95 = df.quantile(0.05), df.quantile(0.25), df.quantile(0.5), df.quantile(0.75), df.quantile(0.95)
#     # [quantile, opacity, length]
#     quants = [[quant_5, 0.6, 0.16], [quant_25, 0.8, 0.26], [quant_50, 1, 0.36],  [quant_75, 0.8, 0.46], [quant_95, 0.6, 0.56]]
#     # Plot the lines with a loop
#     for i in quants:
#         ax.axvline(i[0], alpha = i[1], ymax = i[2], linestyle = ":")
        
    # X #
    ax.set_xlabel("Pixel Height (m)")
    ax.set_xlim(round(df.max())-5, round(df.max())+5)
    # Y #
    #ax.set_ylim(0, 2)
    #ax.set_yticks([])
    ax.set_ylabel("Frequency")
    
    
    # Annotations
#     ax.text(quant_5-.1, 0.17, "5th", size = 10, alpha = 0.8)
#     ax.text(quant_25-.13, 0.27, "25th", size = 11, alpha = 0.85)
#     ax.text(quant_50-.13, 0.37, "50th", size = 12, alpha = 1)
#     ax.text(quant_75-.13, 0.47, "75th", size = 11, alpha = 0.85)
#     ax.text(quant_95-.25, 0.57, "95th Percentile", size = 10, alpha =.8)
    
    # Overall #
    #ax.grid(False)
    ax.set_title("Building #{}".format(ID))

    # Remove ticks and spines
    #ax.tick_params(left = False, bottom = False)
    #for ax, spine in ax.spines.items():
     #   spine.set_visible(False)
        
    plt.show()

#### Add NYKO3 id

In [ ]:
buildings = gpd.read_file(final_buildings + '.shp')
buildings.drop(columns = 'NYKO3', inplace = True)

nyko3 = gpd.read_file(os.path.join(stad_source, 'NYKO3.shp'))
nyko3 = nyko3.loc[:, ['NYKO3', 'NAMN', 'Shape_Leng', 'Shape_Area', 'geometry']]
nyko3 = nyko3.to_crs('epsg:3006')

centroids = buildings.copy()
centroids.loc[:, 'geometry'] = buildings.loc[:'geometry'].centroid
centroids_in_nyko = gpd.sjoin(centroids, nyko3, op='within', how = 'left')

centroids_in_nyko['geometry'] = buildings['geometry']
columns_to_keep = list(buildings) + ['NYKO3']
buildings = centroids_in_nyko[columns_to_keep]
buildings['NYKO3'] = buildings['NYKO3'].astype(str)

# save file
buildings.to_file(final_buildings + '.shp')
buildings.to_excel(final_buildings + '.xlsx')
buildings = buildings.to_crs(epsg=4326)
buildings.to_file(final_buildings + '.geojson', driver='GeoJSON')

#### Check for duplicated records and keep only one

In [ ]:
# check if any record is duplicated; returns all rows 
check_duplication(buildings, drop = False, inplace = True, ignore_index = True)   

In [ ]:
check_duplication(buildings, column = 'BuildingID')  

# Since there are duplicated buildingID, a new one will be created.
buildings['BuildingID'] = range(1, 1 + len(buildings))
#buildings['BuildingID'] = buildings.index + 1
#buildings.drop(columns = 'BuildingID', inplace = True)
check_duplication(buildings, column = 'BuildingID')  

In [ ]:
len(buildings)

# Properties data

In [ ]:
# copy data from data folder to ArcGIS connected folder
for files in os.listdir(source):
    name = os.path.splitext(os.path.basename(files))
    if (raw_properties in files)&(final_properties+name[1] not in os.listdir(destination)):
        shutil.copy(os.path.join(source, files), os.path.join(destination, files))
        
# rename the files in the destination folder
for files in os.listdir(destination):
    name = os.path.splitext(os.path.basename(files))
    if (raw_properties in files)&(final_properties+name[1] not in os.listdir(destination)):
        name = os.path.splitext(os.path.basename(files))
        os.rename(os.path.join(destination, files), os.path.join(destination, final_properties+name[1]))

In [ ]:
# set the destination as the directory
os.chdir(destination)

In [ ]:
#################### load data to process
properties = gpd.read_file(final_properties+'.shp')

In [ ]:
# Drop entries with FNR = 0
# properties = properties[properties['FNR_FDS']!='0']
len(properties)

In [ ]:
check_duplication(properties, drop = False)   

In [ ]:
properties.columns

In [ ]:
#### look for duplicates because of value 2 in YTKVAL where a property geometry can have a duplicate geometrically and 
#### in some fields
#properties.drop(columns=['FNR_FDS', 'OBJEKT_ID','FASTIGHET', 'OMRTYP','EXTERNID'], inplace = True)

#### Drop duplicated geometrical properties
# convert to wkb
properties["geometry"] = properties["geometry"].apply(lambda geom: geom.wkb)
check_duplication(properties, column = 'geometry', drop = False)   

#### A number of properties have the same geometries; keep only the last updated one

In [ ]:
properties = properties.sort_values(by = 'ADAT', ascending = False).drop_duplicates(["geometry"], keep = 'first')
# convert back to shapely geometry
properties["geometry"] = properties["geometry"].apply(lambda geom: shapely.wkb.loads(geom))

#### However, some records have the same FNR_FDS field, or OBJECKT_ID. That is because the file is property areas so a property may have many areas. Create new uuid

In [ ]:
# delete/rename propeties shapefiles columns
# get properties attributes only
attributes_properties = attributes[attributes['File']=='riks']
updated_columns = []
drop_columns = []
for col in properties.columns:
    j = attributes_properties['Attribute'].tolist().index(col)
    updated_columns.append(attributes_properties['NewName'].tolist()[j])
drop_columns = [col for col in properties.columns if updated_columns[properties.columns.tolist().index(col)] == 'REMOVE']
updated_columns = [i for i in updated_columns if i != 'REMOVE']
properties.drop(drop_columns, inplace=True, axis=1)
properties.columns = updated_columns

In [ ]:
# Since there are duplicated propertyID, a new one will be created.
#properties['PropertyID'] = properties.index + 1
properties['PropertyID'] = range(1, 1 + len(properties))

#### Add more info from Lantmäteriet table

In [ ]:
# set the table source as the directory
os.chdir(table_source)
raw_info = 'VESHBYGG_43O.xlsx'
sheetname = 'Data'
# read file
info_table = pd.read_excel(raw_info, sheet_name = sheetname)
# use deep copy to reduce loading time of the data especially when debugging the code
info = copy.deepcopy(info_table)

In [ ]:
info['FNR'] = info['FNR'].astype(str)

In [ ]:
info = info[['FNR','YTABOST','YTABI','YTAVARDE','BYGGAR']]
info.columns = ['FNR','Boyta', 'Biarea', 'TotalArea', 'Year']

In [ ]:
properties = properties.merge(info,how='left', on = 'FNR')

#### ADD NYKO3 id

In [ ]:
# properties = gpd.read_file(final_properties + '.shp')
# properties.drop(columns = 'NYKO3', inplace = True)
# properties = properties.to_crs('epsg:3006')

nyko3 = gpd.read_file(os.path.join(stad_source, 'NYKO3.shp'))
nyko3 = nyko3.loc[:, ['NYKO3', 'NAMN', 'Shape_Leng', 'Shape_Area', 'geometry']]
nyko3 = nyko3.to_crs('epsg:3006')

centroids = properties.copy()
centroids.loc[:, 'geometry'] = properties.loc[:'geometry'].centroid
centroids_in_nyko = gpd.sjoin(centroids, nyko3, op='within', how = 'left')

centroids_in_nyko['geometry'] = properties['geometry']
columns_to_keep = list(properties) + ['NYKO3']
properties = centroids_in_nyko[columns_to_keep]
properties['NYKO3'] = properties['NYKO3'].astype(str)

# save file
# properties.to_file(final_properties + '.shp')
# properties.to_excel(final_properties + '.xlsx')
# properties = properties.to_crs(epsg=4326)
# properties.to_file(final_properties + '.geojson', driver='GeoJSON')

# Addresses data

In [ ]:
# set the table source as the directory
os.chdir(table_source)
#################### load data to process
addresses = gpd.read_file(final_addresses+'.shp')

In [ ]:
# select only addresses within properties extent
polygon = extent.geometry[0]
addresses = addresses[addresses.within(polygon)]
len(addresses)

In [ ]:
addresses.columns

In [ ]:
# delete/rename addresses shapefiles columns
# get addresses attributes only
attributes_addresses = attributes[attributes['File']=='ADRPL_90A']
updated_columns = []
drop_columns = []
for col in addresses.columns:
    j = attributes_addresses['Attribute'].tolist().index(col)
    updated_columns.append(attributes_addresses['NewName'].tolist()[j])
drop_columns = [col for col in addresses.columns if updated_columns[addresses.columns.tolist().index(col)] == 'REMOVE']
updated_columns = [i for i in updated_columns if i != 'REMOVE']
addresses.drop(drop_columns, inplace=True, axis=1)
addresses.columns = updated_columns

In [ ]:
#### Drop duplicated geometrical addresses
# convert to wkb
addresses["geometry"] = addresses["geometry"].apply(lambda geom: geom.wkb)
check_duplication(addresses, drop = False)
# convert back to shapely geometry
addresses["geometry"] = addresses["geometry"].apply(lambda geom: shapely.wkb.loads(geom))

Add Block and Enhet fields from REGENH tables

In [ ]:
os.chdir(table_source)
# add Block and Enhet 
blocks_pd = []
# get filenames
for s in ['A','B','C']:
    addresses_file = 'REGENH_01%s.xlsx'%s
    sheetname = 'Data'
    # read file
    addresses_pd = pd.read_excel(addresses_file, sheet_name = sheetname)
    blocks_pd.append(addresses_pd)
blocks = pd.concat(blocks_pd)

In [ ]:
# delete/rename blocks shapefiles columns
# get blocks attributes only
attributes_blocks = attributes[attributes['File']=='REGENH_01']
updated_columns = []
drop_columns = []
for col in blocks.columns:
    j = attributes_blocks['Attribute'].tolist().index(col)
    updated_columns.append(attributes_blocks['NewName'].tolist()[j])
drop_columns = [col for col in blocks.columns if updated_columns[blocks.columns.tolist().index(col)] == 'REMOVE']
updated_columns = [i for i in updated_columns if i != 'REMOVE']
blocks.drop(drop_columns, inplace=True, axis=1)
blocks.columns = updated_columns

In [ ]:
# merge addresses and blocks
addresses = addresses.merge(blocks,how='left', left_on='UUIDREGENH', right_on='UUID')
addresses.drop(columns = ['UUIDREGENH','UUID'], inplace = True)

In [ ]:
# check coordinates systems and porject if necessary
print('Addresses CRS: ', addresses.crs)
#addresses  = addresses.to_crs('epsg:3006')
#print('Addresses CRS: ', addresses.crs)
#addressesShp.plot()

# Save files

### Save properties shapefile

In [ ]:
# set the destination as the directory
os.chdir(destination)

In [ ]:
properties.columns

In [ ]:
properties['FNR'] = properties['FNR'].astype(str)

In [ ]:
properties.Commune.unique()

In [ ]:
check_duplication(properties.loc[properties['Commune']=='1980', :], column = 'FNR', drop = False, 
                                     keep = 'first', inplace = True, ignore_index = True)

In [ ]:
# when done move it to Properties section above
buildings = gpd.read_file(final_buildings + ".shp")
properties_duplicated = properties.loc[properties['FNR'].duplicated(), :]
buildings = buildings.loc[buildings['PropertyID'].isin(properties_duplicated['PropertyID'].unique().tolist()), :]
buildings.to_file('testing.shp')

In [ ]:
df_obj = properties.select_dtypes(['object'])
properties[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
# write the new properties shapefile
properties.to_file(final_properties + '.shp')
properties.to_excel(final_properties + '.xlsx')
properties = properties.to_crs(epsg=4326)
properties.to_file(final_properties + '.geojson', driver='GeoJSON')

### Save buildings shapefile

In [ ]:
len(buildings)

In [ ]:
buildings.columns

In [ ]:
# convert type of id to float to make it possible to link addresses id to buildings id as database tables. Otherwise
# nan id won't be accepted
# buildings['BuildingID'] = buildings['BuildingID'].astype('float')

In [ ]:
### add street address to buildings. 
# addressesInbuildings = gpd.sjoin(addresses, buildings, op='within')
# addressesInbuildings.sort_index(inplace=True)
# addressesInbuildings = addressesInbuildings.drop(['index_right'], axis=1)

# check_duplication(addressesInbuildings, column = 'BuildingID', drop = True, 
#                                      keep = 'first', inplace = True, ignore_index = True)

# # add street, zip code, number to buildings
# columns_to_add = ['Street','Municipal','PostNb','PostOffice', 'BuildingID']# 'AdrPlace','AdrPlaceTp','County','Commune','AdrAreaTp',
# buildingsWithaddresses = buildings.merge(addressesInbuildings[columns_to_add],how='left', left_on='BuildingID', right_on='BuildingID')

In [ ]:
properties = properties.to_crs(epsg=3006)

In [ ]:
### because of limitation in the spatial join, we use the buildings centroids 
buildingsWithaddresses = buildings.copy()

buildings_centroids = buildingsWithaddresses.copy()
buildings_centroids['geometry'] =  buildings_centroids['geometry'].centroid

buildingsInproperties = gpd.sjoin(buildings_centroids, properties, op='within')
#buildingsInproperties.reset_index(inplace=True, drop=True)
buildingsInproperties.sort_index(inplace = True)
#buildingsInproperties['Commune'] = buildingsInproperties['Commune_left'] 
#buildingsInproperties['Municipal'] = buildingsInproperties['Municipal_left'] 
buildingsInproperties = buildingsInproperties.drop(['index_right'], axis=1)

In [ ]:
# Keep all addresses columns with the building ID column
columns_to_keep = list(buildingsWithaddresses.columns) + ['PropertyID']
buildingsInproperties = buildingsInproperties[columns_to_keep]

In [ ]:
buildingsInproperties.crs

In [ ]:
buildingsInproperties["geometry"] = buildingsInproperties["geometry"].apply(lambda geom: geom.wkb)

In [ ]:
buildingsInproperties["geometry"] = buildingsInproperties["geometry"].apply(lambda geom: geom.wkb)
check_duplication(buildingsInproperties, column = 'geometry', drop = False)
buildingsInproperties["geometry"] = buildingsInproperties["geometry"].apply(lambda geom: shapely.wkb.loads(geom))

In [ ]:
# return the polygon geomtry instead of centroids
buildingsInproperties['geometry'] = buildings['geometry']

In [ ]:
len(buildingsInproperties)

In [ ]:
# save file
buildingsInproperties.to_file(final_buildings + '.shp')
buildingsInproperties.to_excel(final_buildings + '.xlsx')
buildingsInproperties = buildingsInproperties.to_crs(epsg=4326)
buildingsInproperties.to_file(final_buildings + '.geojson', driver='GeoJSON')
# save files to database
# write_to_postgis(buildingsInproperties, final_buildings, engine, primary_keys = ['BuildingID'], 
#                  foreign_keys = ['PropertyID'], reference_keys = ['PropertyID'], 
#                  reference_table_name = [final_properties])

In [ ]:
buildingsInproperties= gpd.read_file(final_buildings + '.shp')

In [ ]:
# save file
df_obj = buildingsInproperties.select_dtypes(['object'])
buildingsInproperties[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
#buildingsInproperties.drop(columns = 'FNR', inplace = True)
#buildingsInproperties.to_file(final_buildings + '.shp')
#buildingsInproperties.to_excel(final_buildings + '.xlsx')
buildingsInproperties = buildingsInproperties.to_crs(epsg=4326)
buildingsInproperties.to_file(final_buildings + '.geojson', driver='GeoJSON')

### Save addresses shapefile

In [ ]:
buildings = gpd.read_file(final_buildings + '.shp')

In [ ]:
# import pyproj
# pyproj.datadir.set_data_dir(r'C:\Users\akm03\.conda\envs\new_env_geo\share\proj')

In [ ]:
addressesInproperties = gpd.sjoin(addresses, properties[['PropertyID','geometry']], op='within')

In [ ]:
# spatial join with properties to link each address to one. Some addresses are outside the building polygon and could
# be discarded 
addressesInproperties = gpd.sjoin(addresses, properties[['PropertyID','geometry']], op='within')
addressesInproperties.sort_index(inplace=True)
addressesInproperties = addressesInproperties.drop(['index_right'], axis=1)

In [ ]:
addressesInproperties["geometry"] = addressesInproperties["geometry"].apply(lambda geom: geom.wkb)
check_duplication(addressesInproperties, column = 'geometry', drop = False)
# convert back to shapely geometry
addressesInproperties["geometry"] = addressesInproperties["geometry"].apply(lambda geom: shapely.wkb.loads(geom))

In [ ]:
# addressesInproperties = addressesInproperties.to_crs(epsg=4326)
# how = 'left' to keep all records even those who are not spatial joined.
addressesInbuildings = gpd.sjoin(addressesInproperties, buildings[['BuildingID','geometry']], 
                                 op='within', how='left')
addressesInbuildings.sort_index(inplace=True)
# addressesInbuildings = addressesInbuildings.drop(['index_right', 'FNR_x', 'FNR_y'], axis=1)

In [ ]:
len(addressesInproperties), len(addresses), len(addressesInbuildings)

In [ ]:
check_duplication(addressesInbuildings, drop = True)

In [ ]:
addressesInbuildings['BuildingID'] = addressesInbuildings['BuildingID_right']
addressesInbuildings = addressesInbuildings.drop(['BuildingID_left', 'Temp', 'index_right', 'BuildingID_right'], axis=1)

In [ ]:
addressesInbuildings = gpd.read_file(final_addresses + '.geojson')

In [ ]:
addressesInbuildings = addressesInbuildings.rename(columns = {'address_id':'AddressID'})
addressesInbuildings = addressesInbuildings[['TableType','Street', 'AdrPlace', 'PostNb', 'PostOffice',
                                             'Municipal', 'TRAKT', 'BLOCK', 'ENHET', 
                                              'County','Commune', 'AdrAreaTp', 
                                             'AddressID', 'PropertyID', 'BuildingID','geometry']]

In [ ]:
addressesInbuildings['BuildingID'] = addressesInbuildings['BuildingID'].fillna(0).astype(int)
addressesInbuildings['PostNb'] = addressesInbuildings['PostNb'].astype(int)

In [ ]:
addressesInbuildings = gpd.GeoDataFrame(addressesInbuildings, geometry=addressesInbuildings['geometry'])

In [ ]:
# # write the new addresses shapefile
# addressesInbuildings = addressesInbuildings.to_crs(epsg=3006)
addressesInbuildings.to_file(final_addresses + '.shp')
addressesInbuildings.to_excel(final_addresses + '.xlsx')

addressesInbuildings = addressesInbuildings.to_crs(epsg=4326)
addressesInbuildings.to_file(final_addresses + '.geojson', driver='GeoJSON')
# save files to database
# write_to_postgis(addressesInbuildings, final_addresses, engine, primary_keys = ['AddressID'], 
#                  foreign_keys = ['BuildingID','PropertyID'], reference_keys = ['BuildingID','PropertyID'], 
#                  reference_table_name = [final_buildings, final_properties])